# 1. Capture images for chessboard

In [ ]:
import cv2

cap = cv2.VideoCapture(1)
num = 0

while cap.isOpened():
    success, img = cap.read()
    k = cv2.waitKey(5)
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('images/' + str(num) + '.png', img)
        print("image saved!")
        num += 1
    cv2.imshow('Img',img)

cap.release()
cv2.destroyAllWindows()

# 2. Check that corners are found

If chessboard is (6,6) so corners are (5,5) 

In [ ]:
import cv2 as cv
import glob

chessboardSize = (5,4)  #corners 
images = glob.glob('images/*.png')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)
    print(fname, " -> ", ret)

    if ret:
        cv.drawChessboardCorners(img, chessboardSize, corners, ret)
    cv.imshow('Check', img)
    cv.waitKey(500)

cv.destroyAllWindows()


# 3. Camera Calibration to calculate desired parameters

In [ ]:
import numpy as np
import cv2 as cv
import glob

chessboardSize = (5,4)
frameSize = (640,480)

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1,2)

objPoints = []  
imgPoints = []  

images = glob.glob('images/*.png')
for image in images:
    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(
        gray, chessboardSize, 
        cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE
    )

    if ret:
        objPoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgPoints.append(corners2)

cv.destroyAllWindows()

# --- Camera calibration ---
ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objPoints, imgPoints, frameSize, None, None)

print("Camera Calibrated: ", ret)
print("\nCamera Matrix:\n", cameraMatrix)
print("\nDistortion Parameters:\n", dist)

mean_error = 0
for i in range(len(objPoints)):
    imgPoints2, _ = cv.projectPoints(objPoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgPoints[i], imgPoints2, cv.NORM_L2)/len(imgPoints2)
    mean_error += error
print("\nTotal error: {}".format(mean_error/len(objPoints)))

# --- Try extrinsic on first valid image ---
for image in images:
    test_img = cv.imread(image)
    gray = cv.cvtColor(test_img, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(
        gray, chessboardSize, 
        cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE
    )
    if ret:
        _, rvec, tvec = cv.solvePnP(objp, corners, cameraMatrix, dist)
        print(f"\nExtrinsic parameters for {image}:")
        print("Rotation Vector:\n", rvec)
        print("Translation Vector:\n", tvec)

        R, _ = cv.Rodrigues(rvec)
        print("\nRotation Matrix:\n", R)

        extrinsic = np.hstack((R, tvec))
        print("\nExtrinsic Matrix [R|t]:\n", extrinsic)

        proj_points, _ = cv.projectPoints(objp, rvec, tvec, cameraMatrix, dist)
        error = cv.norm(corners, proj_points, cv.NORM_L2) / len(proj_points)
        print("\nReprojection error on this image: ", error)

        break
